# Qwen2-VL-7B-Instruct

In [1]:
! python -m pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 123.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.2
    Uninstalling pip-24.2:
      Successfully uninstalled pip-24.2


In [2]:
! pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-_i5gd5gj
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-_i5gd5gj
  Resolved https://github.com/huggingface/transformers to commit c8c8dffbe45ebef0a8dba4a51024e5e5e498596b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 97.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 192.3 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.48.0.dev0-py3-none-any.whl size=10185493 sha256=9394e88be5e7412f7ea6f29fdc344af214d74a6b5d3f269446a1cd0d46447e47
  Stored in directory: /tmp/pip-ephem-wheel-cache-jruz3gws/wheels/04/a3/f1/b88775f8e1665827525b19ac7590250f1038d947067beba9fb
Successfully built transformers


In [3]:
! pip install -r qwen_requirement.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 204.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 164.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 179.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 124.0 MB/s eta 0:00:00a 0:00:01


In [4]:
! pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 167.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.7.1.post4-cp311-cp311-linux_x86_64.whl size=187423065 sha256=60b950e158e9a0b50b689b2d8c84af3798cad36eec2b92ef56dc3a7ca029e4e6
  Stored in directory: /root/.cache/pip/wheels/c3/05/97/a93f42c29fd0fd252c48e8582e485a5745ada372bd2161a07a
Successfully built flash-attn


Need to restart kernel after installing library

In [1]:
import os
import torch
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, classification_report
from tqdm import tqdm
import json


# Open log file
log_path = './results/qwen2-7B-V-Instruct/'
os.makedirs(log_path, exist_ok=True)
log_file = os.path.join(log_path, 'prediction_log.txt')
log_file = open(log_file, 'w')


# Load data
df_table_prd = pd.read_csv('../LMM_sewerML/results/df_table_prd.csv')
df_table_dsc = pd.read_csv('../LMM_sewerML/results/df_table_dsc.csv')
image_dir = '../LMM_sewerML/images'


# Load Qwen model and processor
model_id = "Qwen/Qwen2-VL-7B-Instruct"
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    device_map="auto",
)
processor = AutoProcessor.from_pretrained(model_id)


# Define the prompt template
prompt_template = """
You are a virtual sewer technician with the capability to analyze images from CCTV cameras taken inside sewer pipes.
Your task is to examine each image and provide a concise, yet accurate, summary for retrieval. 
After summarizing, you must classify the image as defect types.
While providing the summary, remember the following guidelines:
1) Provide a general overview of the image that you see, describing important elements such image clarity, lighting conditions, type of pipe (concrete, PVC, ...), presence of water.
2) Check for defects in the sewer pipes in the image.
3) Pipes in good condition usually show a smooth, unbroken surface, no visible signs of damage like cracks or collapses, and an absence of blockages such as roots.
4) On the other hand, you can have the following defects:
4a) Cracks, Breaks, and Collapses (RB): Identify visible cracks along the pipe, instances where the pipe has fractured or completely broken apart, and areas where the pipe has collapsed.
This includes longitudinal cracks, circumferential breaks, and complete structural failures that compromise the integrity of the sewer system.
4b) Surface Damage (OB): Detect areas of the pipe's interior that exhibit signs of wear, erosion, or damage on the surface.
This includes minor scratches, pitting, scaling, or any form of deterioration that affects the pipe's surface but does not necessarily penetrate deeply into the structure.
4c) Production Error (PF): Identify defects that originated during the pipe's manufacturing process, such as inconsistent pipe thickness, improper joint alignment, or material imperfections.
These are flaws that were introduced before installation and could potentially affect the pipe's performance or longevity.
4d) Deformations (DE): Recognize any alterations in the shape of the pipe, such as bending, sagging, or bulging, that indicate a deformation.
This includes both minor deformations that may affect flow efficiency and major deformations that threaten the pipe's structural integrity.
4e) Roots (RO): Detect the presence of roots infiltrating the sewer pipe, whether through joints, cracks, or other vulnerabilities.
This involves identifying both the initial stages of root intrusion and the more advanced stages where roots have significantly obstructed the pipe.
5) Additional considerations while analyzing the images: do not consider blurred text or user-defined circled areas in the images.
6) You will always try to describe the image that you see. Provide the output in JSON format as follows:

{ "DESCRIPTION": "<Description of the image that you see>", "CODE": "<Defect Code>"}

Note: The "CODE" can be selected from "RB", "OB", "PF", "DE", "RO". If no defect is detected, set "CODE" to "NoDefect".
"""

# Store predictions and actual labels
predictions = []
actual_labels = []
descriptions = []


# Add tqdm progress bar
for idx, row in tqdm(df_table_prd.iterrows(), total=len(df_table_prd), desc="Processing images"):
    img_id = row['img_id']
    ground_truth = row['defect_type']
    img_path = os.path.join(image_dir, img_id)
    
    # Check if image file exists
    if not os.path.exists(img_path):
        log_file.write(f"Image {img_id} not found. Skipping.\n")
        continue

    # Load image
    # image = Image.open(img_path)

    # Prepare input for Qwen
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": img_path},
                {"type": "text", "text": prompt_template},
            ],
        }
    ]
    
    # Prepare processor inputs
    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, _ = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        padding=True,
        return_tensors="pt",
    ).to("cuda")

    # Generate output
    generation_args = {
        "max_new_tokens": 1024,
    }
    outputs = model.generate(**inputs, **generation_args)

    generated_ids_trimmed = outputs[:, inputs.input_ids.shape[1]:]
    response = processor.decode(generated_ids_trimmed[0], skip_special_tokens=True)
    print(response)

    if response.startswith("```"):
        cleaned_response_text = response.split('\n', 1)[-1].rsplit('\n', 1)[0].strip()
    else:
        cleaned_response_text = response.strip()
    print(cleaned_response_text)
    # Extract defect prediction and description from model output
    try:
        response_json = json.loads(cleaned_response_text)
        description = response_json.get("DESCRIPTION", "No description provided")
        predicted_defect = response_json.get("CODE", "NoDefect")
    except json.JSONDecodeError:
        print(f"Invalid JSON response for Image ID {img_id}: {response}\n")
        log_file.write(f"Invalid JSON response for Image ID {img_id}: {response}\n")
        description = "Error in JSON response"
        predicted_defect = "Error"

    # Append results to lists
    predictions.append(predicted_defect)
    actual_labels.append(ground_truth)
    descriptions.append(description)

    # Log intermediate outputs
    print(f"Image ID: {img_id} | Predicted: {predicted_defect} | Ground Truth: {ground_truth}\n")
    log_file.write(f"Image ID: {img_id} | Predicted: {predicted_defect} | Ground Truth: {ground_truth}\n")


# Calculate evaluation metrics
accuracy = accuracy_score(actual_labels, predictions)
f1 = f1_score(actual_labels, predictions, average='weighted')
report = classification_report(actual_labels, predictions)

log_file.write(f"\nAccuracy: {accuracy:.4f}\n")
log_file.write(f"F1 Score: {f1:.4f}\n")
log_file.write("\nClassification Report:\n")
log_file.write(report)


# Close log file
log_file.close()


# Save results to CSV files
df_table_prd['qwen-vl-7B'] = predictions
df_table_prd.to_csv(log_path + 'df_table_prd.csv', index=False)

df_table_dsc['qwen-vl-7B'] = descriptions
df_table_dsc.to_csv(log_path + 'df_table_dsc.csv', index=False)


`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Processing images:   0%|          | 1/200 [00:03<10:03,  3.03s/it]

```json
{
  "DESCRIPTION": "The image shows a concrete sewer pipe with a visible crack running through it. The pipe appears to be in a deteriorated state, with signs of wear and erosion on the surface. There are no signs of water or liquid inside the pipe.",
  "CODE": "RB"
}
```
{
  "DESCRIPTION": "The image shows a concrete sewer pipe with a visible crack running through it. The pipe appears to be in a deteriorated state, with signs of wear and erosion on the surface. There are no signs of water or liquid inside the pipe.",
  "CODE": "RB"
}
Image ID: 00617545.png | Predicted: RB | Ground Truth: RB



Processing images:   1%|          | 2/200 [00:05<09:43,  2.95s/it]

...

Processing images: 100%|█████████▉| 199/200 [08:00<00:02,  2.34s/it]

```json
{
  "DESCRIPTION": "The image shows a section of a sewer pipe with a smooth surface. There are no visible cracks, breaks, or collapses. The lighting is adequate, and the pipe appears to be made of concrete. There is no water present in the pipe.",
  "CODE": "NoDefect"
}
```
{
  "DESCRIPTION": "The image shows a section of a sewer pipe with a smooth surface. There are no visible cracks, breaks, or collapses. The lighting is adequate, and the pipe appears to be made of concrete. There is no water present in the pipe.",
  "CODE": "NoDefect"
}
Image ID: 00835200.png | Predicted: NoDefect | Ground Truth: DE



Processing images: 100%|██████████| 200/200 [08:03<00:00,  2.42s/it]

```json
{
  "DESCRIPTION": "The image shows a sewer pipe with a smooth surface, no visible cracks, breaks, or collapses. There is no water present in the pipe, and the lighting conditions are adequate. The pipe appears to be in good condition.",
  "CODE": "NoDefect"
}
```
{
  "DESCRIPTION": "The image shows a sewer pipe with a smooth surface, no visible cracks, breaks, or collapses. There is no water present in the pipe, and the lighting conditions are adequate. The pipe appears to be in good condition.",
  "CODE": "NoDefect"
}
Image ID: 00507208.png | Predicted: NoDefect | Ground Truth: PF




/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
